# Train and Serve a watsonx.ai Custom Model

## Part 2: Deploy the custom-trained model to SAP AI Core

This two-part mini-series demonstrates a use-case, in which you train a model in [IBM Watson Studio](https://www.ibm.com/products/watson-studio) with data from [SAP Datasphere](https://help.sap.com/docs/SAP_DATASPHERE) and deploy it to [SAP AI Core](https://help.sap.com/docs/sap-ai-core). You could independently run the two notebooks provided without major modifications, if you only want to train or serve a similar model.

You will learn:

- How to access datasets stored in SAP Datasphere
- How to train a model in Watson Studio
- How to sync a trained model to SAP AI Core
- How to create a workflow to serve a model
- How to deploy a model and use it for online inferencing

### Prerequisites

- [SAP Businness Technology Platform (BTP)](https://help.sap.com/docs/btp) services and applications
    - [SAP Datasphere](https://discovery-center.cloud.sap/serviceCatalog/sap-datasphere?region=all)
    - [SAP AI Core](https://discovery-center.cloud.sap/serviceCatalog/sap-ai-core?region=all)
    - [SAP AI Launchpad](https://discovery-center.cloud.sap/serviceCatalog/sap-ai-launchpad?region=all)
    - [Object Store](https://help.sap.com/docs/object-store)
- Access to a [project](https://www.ibm.com/docs/en/watsonx?topic=projects) in watsonx
- Your entitlement key from the [container software library](https://myibm.ibm.com/products-services/containerlibrary) to access the IBM Entitled Registry
- A [GitHub repository](https://docs.github.com/en/get-started/quickstart/create-a-repo) and [personal access token (PAT)](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens) to access the repository

## Table of Contents

1.  [Step 1: Set up SAP AI Core SDK to connect with SAP AI Core](#aicore_sdk)

1.  [Step 2: Onboard GitHub to SAP AI Core](#onboard)

1.  [Step 3: Create a Docker registry secret for IBM Entitled Registry](#entitled_registry)

1.  [Step 4: Load the custom-trained model](#load_model)

1.  [Step 5: Upload the custom-trained model to Object Store](#upload_model)

1.  [Step 6: Create a workflow to serve the model](#create_workflow)

1.  [Step 7: Register the model as an artifact](#register_artifact)

1.  [Step 8: Create configuration for deployment](#create_configuration)

1.  [Step 9: Start the deployment](#start_deployment)

1.  [Step 10: Make a prediction](#make_prediction)

1.  [Step 11: Stop the deployment](#stop_deployment)

1.  [Summary](#summary)

<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with NLP Runtime 23.1 on Python 3.10 XS environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)
<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

In [1]:
# The code was removed by Watson Studio for sharing.

Before you begin in part 2, you'll need to use your own credentials to replace what's in the examples below. To avoid sharing them accidentally, copy the following code block into a new code cell, and make it a hidden cell by adding `# @hidden_cell` to the top.

```
# Set your SAP AI Core credential
client_id = '<your_client_id>'
client_secret = '<your_client_secret>'
base_url = '<ai_api_url>' + '/v2'
auth_url = '<url>' + '/oauth/token'

# Set your Object Store credential
access_key_id = '<your_access_key_id>'
secret_access_key = '<your_secret_access_key>'
bucket = '<your_bucket>'
endpoint = '<host>'
region = '<region>'

# Set the GitHub credential
github_cred_name = '<any_unique_name_for_the_git_repository_credential>'
github_repo_url = 'https://github.com/<your_username>/<repo_name>'
github_username = '<your_username>'
github_password = '<your_personal_assess_token>'

# Set the Docker Registry credential with your IBM Entitlement Key
entitled_registry_credential = "{\"auths\":{\"cp.icr.io\":{\"username\":\"cp\",\"password\":\"<your_key>\"}}}"
```

In [2]:
# The code was removed by Watson Studio for sharing.

<a id="aicore_sdk"></a>
### Step 1: Set up SAP AI Core SDK to connect with SAP AI Core

The SAP AI Core SDK is a Python-based SDK that lets you access SAP AI Core using Python methods and data structures. It provides tools that help you to manage your scenarios and workflows in SAP AI Core. The SAP AI Core SDK can be used to interact with SAP AI Core. It provides access to all public lifecycle and administration APIs.

The command line tool `pip` is the Python package installer. You can use `pip` to install the SAP AI Core SDK for Python:

In [3]:
# Install SAP AI Core SDK
%pip install ai-core-sdk

Note: you may need to restart the kernel to use updated packages.


Connect with SAP AI Core with your AI Core credential:

In [4]:
# Load AI Core Client Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create a client connection
ai_core_client = AICoreV2Client(
    base_url = base_url,
    auth_url = auth_url,
    client_id = client_id,
    client_secret = client_secret
)

<a id="onboard"></a>
### Step 2: Onboard GitHub to SAP AI Core

You can use your own git repository to version control your SAP AI Core templates. The GitOps onboarding to SAP AI Core instances involves setting up your git repository and synchronizing your content.

On-board a new GitHub repository:

In [5]:
ai_core_client.repositories.create(
    name = github_cred_name,
    url = github_repo_url,
    username = github_username,
    password = github_password
)

Execute the command below to list the GitHub repositories connected to your SAP AI Core:

In [6]:
# Execute the command below to list the GitHub repositories connected to your SAP AI Core.
response = ai_core_client.repositories.query()
print(response.count)

2


<a id="entitled_registry"></a>
### Step 3: Create a Docker registry secret for IBM Entitled Registry

IBM Entitled Registry stores Watson NLP runtime container image required to serve the custom-trained model. The credentials for your Docker registries are managed using secrets.

Create a Docker registry secret for connecting SAP AI Core to the IBM Entitled Registry:

In [7]:
response = ai_core_client.docker_registry_secrets.create(
    name = "ibm-entitled-registry-secret",
    data = {
        ".dockerconfigjson": entitled_registry_credential
    }
)

print(response.__dict__)

{'message': 'secret has been created'}


<a id="load_model"></a>
### Step 4: Load the custom-trained model

You can download the custom-trained model, saved as a project asset by the part 1 notebook, from the Cloud Object Storage (COS) bucket associated with the Watson Studio instance, into the local file system of your Watson Studio Environment runtime.

Create directory in the local file system:

In [8]:
import os

# Set some variables
home = os.environ['HOME']
parent_dir = os.path.join(home, 'models')
parent_dir

# Create a directory for saving models
!mkdir -p {parent_dir}

Download the custom model into the directory you just created:

In [9]:
# Import Watson NLP library
import watson_nlp

# Download the model
wslib.download_file('msds_classification_model', file_name=os.path.join(parent_dir, 'msds_classification_model'))

{'file_name': '/home/wsuser/models/msds_classification_model',
 'summary': ['loaded data', 'saved to file']}

Load the model from the local file system path:

In [10]:
# Load the model
custom_model = watson_nlp.load(os.path.join(parent_dir, 'msds_classification_model'))

Run the model on the input text to make sure it works as expected:

In [11]:
# Run the model on the input text
custom_model.run('DEPENDS ON CHEMICAL COMPOSITION; FISSION PRODUCTS SAFETY GLASSES EYE WASH STATION. APRONS. SPECIAL IMPERVIOUS CLOTHING. IMPERVIOUS SCBA IF INVOLVED IN FIRE, OTHERWISE GAS MASK. PROVIDE MECHAN(GEN/LOCAL EXHAUST)VENT TO MAINTN <TLV	')

{
  "classes": [
    {
      "class_name": "WEAR CHEMICAL PROTECTIVE SUIT WITH SELF CONTD BRTHG APP.",
      "confidence": 0.9815407991409302
    },
    {
      "class_name": "WEAR NIOSH/MSHA APPROVED SCBA & FULL PROTECTIVE EQUIPMENT .",
      "confidence": 0.007698021829128265
    },
    {
      "class_name": "NONE",
      "confidence": 0.007614371832460165
    },
    {
      "class_name": "WEAR NIOSH/MSHA APPROVED SCBA AND FULL PROTECTIVE EQUIPMENT .",
      "confidence": 0.00588197773322463
    },
    {
      "class_name": "FIRE FIGHTERS SHOULD USE NIOSH APPROVED SCBA & FULL PROTECTIVE EQUIPMENT WHEN FIGHTING CHEMICAL FIRE. USE WATER SPRAY TO COOL NEARBY CONTAINERS EXPOSED TO FIRE.",
      "confidence": 0.005488577764481306
    },
    {
      "class_name": "SELF-CONTAINED BREATHING APPARATUS.",
      "confidence": 0.0039141918532550335
    },
    {
      "class_name": "COMBUSTION PRODUCTS OF THIS MATERIAL MAY BE TOXIC. USE A SELF-CONTAINED BREATHING APPARATUS WHEN FIGHTING FIRES INV

<a id="upload_model"></a>
### Step 5: Upload the custom-trained model to Object Store

SAP AI Core supports multiple hyperscaler object stores, such as Amazon S3, OSS (Alicloud Object Storage Service), SAP HANA Cloud, Data Lake and Azure Blob Storage. The connected storage stores your datasets, models and other cache files of the Metaflow Library for SAP AI Core. Storage credentials are managed using secrets. You can create multiple object store secrets.

Create an object store secret to connect your AWS S3 bucket to SAP AI Core. Note that the parameter `path_prefix` should point to the parent folder of the folder where your model is located in AWS S3.

In [12]:
path_prefix = 'watsonx/demo'

response = ai_core_client.object_store_secrets.create(
    resource_group = 'default',
    type = "S3",
    name = "s3-watsonx-demo",
    path_prefix = path_prefix,
    endpoint = endpoint,
    bucket = bucket,
    region = region,
    data = {
        "AWS_ACCESS_KEY_ID": access_key_id,
        "AWS_SECRET_ACCESS_KEY": secret_access_key
    }
)

print(response.__dict__)

{'message': 'secret has been created'}


<div class="alert alert-block alert-info">

<b>Tip:</b>

Depending on your region, your AWS endpoint syntax may differ from the example above. In the event of an error, try this step again with alternative syntax. For available syntaxes, please see the [AWS documentation](https://docs.aws.amazon.com/AmazonS3/latest/userguide/WebsiteEndpoints.html).

</div>

Upload the model to Object Store:

In [13]:
# Install awscli
%pip install --upgrade --user awscli

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Copy the models directory to the S3 bucket
!AWS_ACCESS_KEY_ID={access_key_id} AWS_SECRET_ACCESS_KEY={secret_access_key} {home}/.local/bin/aws --endpoint-url https://{endpoint} s3 cp {parent_dir} s3://{bucket}/{path_prefix}/models --recursive

upload: ../models/msds_classification_model to s3://sap-ai-core-demo-bucket/watsonx/demo/models/msds_classification_model


<a id="create_workflow"></a>
### Step 6: Create a workflow to serve the model

SAP AI Core supports multiple hyperscaler object stores, such as Amazon S3, OSS (Alicloud Object Storage Service), SAP HANA Cloud, Data Lake and Azure Blob Storage. The connected storage stores your dataset, models and other cache files of the Metaflow Library for SAP AI Core. Storage credentials are managed using secrets. You can create multiple object store secrets.

You will create a folder in your GitHub repository connected SAP AI Core, where you will store the workflow (executable). You will then register this folder as an **Application** in SAP AI Core to enable syncing of the workflow as an executable.

You can create multiple **Applications** in SAP AI Core for syncing multiple folders. This helps you organize separate folders for storing workflows YAML files for separate use cases.

1. Create a executable YAML file named `watsonx-demo/serving_executable.yaml` in your GitHub repository connected to SAP AI Core.
2. Edit and execute the code below to create an **Application** and sync the folder `watsonx-demo`
3. Verify your workflow sync status, using the following code

You may use the existing GitHub path which is already tracked synced to your application of SAP AI Core.

**IMPORTANT**: The structure(schemas) of workflows and executables are different for both training and serving in SAP AI Core. For available options for the schemas you must refer to the [official help guide of SAP AI Core](https://help.sap.com/docs/AI_CORE/2d6c5984063c40a59eda62f4a9135bee/8a1f91a18cf0473e8689789f1636675a.html?locale=en-US).

```yaml
apiVersion: ai.sap.com/v1alpha1
kind: ServingTemplate
metadata:
  name: watsonx-demo-serve
  annotations:
    scenarios.ai.sap.com/description: "Watson NLP models"
    scenarios.ai.sap.com/name: "watsonx-demo"
    executables.ai.sap.com/description: "Watson NLP serving executable"
    executables.ai.sap.com/name: "watsonx-demo-serve-exectuable"
  labels:
    scenarios.ai.sap.com/id: "watsonx-demo"
    ai.sap.com/version: "1.0.0"
spec:
  inputs:
    artifacts:
    - name: modelArtifact
  template:
    apiVersion: "serving.kserve.io/v1beta1"
    metadata:
      labels: |
        ai.sap.com/resourcePlan: starter
    spec: |
      predictor:
        imagePullSecrets:
          - name: ibm-entitled-registry-secret
        minReplicas: 1
        maxReplicas: 2
        containers:
        - name: kserve-container
          image: "cp.icr.io/cp/ai/watson-nlp-runtime:1.1.32"
          ports:
            - containerPort: 8080
              protocol: TCP
          resources:
            limits:
              cpu: 2
              memory: 4Gi
            requests:
              cpu: 1
              memory: 2Gi
          env:
            - name: STORAGE_URI
              value: "{{inputs.artifacts.modelArtifact}}"
            - name: ACCEPT_LICENSE
              value: "true"
            - name: LOCAL_MODELS_DIR
              value: "/mnt/models"
```

**Understanding your serving executable**

- You use an input artifacts placeholder `modelArtifact` for your model.
- You use the `starter` computing resource plan with `ai.sap.com/resourcePlan`. To start, using a non-GPU based resource plan for serving (like `starter`) is cost effective. Find out more about available resource plans in [the help portal](https://help.sap.com/docs/AI_CORE/2d6c5984063c40a59eda62f4a9135bee/57f4f19d9b3b46208ee1d72017d0eab6.html?locale=en-US).
- You set the auto scaling of the server with the parameters: `minReplicas` and `maxReplicas`.
- You set the credentials to access the docker registry via `imagePullSecrets`. You must ensure that if you are using a public docker registry that has the file type `docker.io`, your secret points to the URL `https://index.docker.io`. You may delete and recreate the docker registry secret. This will not affect training templates running in parallel.
- You use the placeholder `env` to pass your `inputs` values as environment variables in your Docker image.
- You use the model placeholder value (reference to cloud storage) `STORAGE_URI` through the environment variables. The model files stored in your cloud storage (referenced by the value of your input artifacts placeholder) will be copied to the path `/mnt/models` inside your Docker image.
- You must set the name of the container to `kfserving-container` or `kserve-container`.
- You use the environment variable `LOCAL_MODELS_DIR` to specify the path to the models. By default, Watson NLP runtime will try to load the models from `/app/models`.

Create an `Application` in SAP AI Core:

In [15]:
# Create an Application
response = ai_core_client.applications.create(
    application_name = "watsonx-demo-app",
    revision = "HEAD",
    repository_url = github_repo_url,
    path = "watsonx-demo"
)

print(response.__dict__)

{'id': 'watsonx-demo-app', 'message': 'Application has been successfully created.'}


Verify your workflow sync status:

In [16]:
import time

time.sleep(60)
response = ai_core_client.applications.get_status(application_name = 'watsonx-demo-app')

print(response.__dict__)
print('*'*80)
print(response.sync_ressources_status[0].__dict__)

{'health_status': 'Healthy', 'sync_status': 'Synced', 'message': 'successfully synced (all tasks run)', 'source': <ai_core_sdk.models.application_source.ApplicationSource object at 0x7f1f4c2c8fd0>, 'sync_finished_at': '2024-02-17T23:21:23Z', 'sync_started_at': '2024-02-17T23:21:22Z', 'reconciled_at': '2024-02-17T23:21:23Z', 'sync_ressources_status': [<ai_core_sdk.models.application_resource_sync_status.ApplicationResourceSyncStatus object at 0x7f1f4c2a7940>]}
********************************************************************************
{'name': 'watsonx-demo-serve-23091515', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/watsonx-demo-serve-23091515 created'}


After you workflows are synced, your `Scenario` will be automatically created in SAP AI Core. The name and ID of the scenario will be same as the one mentioned in your workflows. After The syncing, your workflow will be recognized as an executable.

<a id="register_artifact"></a>
### Step 7: Register the model as an artifact

An artifact is a reference to your model files located in AWS S3.

Use the object store secret created in the previous step in the URL field. The URL takes the form `ai://YOUR_OBJECT_STORE_SECERT_NAME/path/to/files`. The `YOUR_OBJECT_STORE_SECERT_NAME` value will be replaced by the `path_prefix` you had used in that object store secret.

The value of `scenario_id` should not already exist in SAP AI Core at this step, but the following code will not create any errors.

In [17]:
from ai_core_sdk.models import Artifact

response = ai_core_client.artifact.create(
    resource_group = 'default',
    name = "watsonx-demo-model",
    kind = Artifact.Kind.MODEL,
    url = "ai://s3-watsonx-demo/models",
    scenario_id = "watsonx-demo",
    description = "watsonx demo models"
)

print(response.__dict__)
artifact_id=response.__dict__['id']

{'id': '67606ea4-b3be-4b36-b09d-8c4c684d87e7', 'message': 'Artifact acknowledged', 'url': 'ai://s3-watsonx-demo/models'}


<div class="alert alert-block alert-warning">
<b>Important:</b>

Make a note of the ID. This ID uniquely identifies your artifact.

</div>

<a id="create_configuration"></a>
### Step 8: Create configuration for deployment

Here are the important pieces of your configuration:

- The `scenario_id` should contain the same value as in your executable.
- The `executable_id` is the name key of your executable.
- The `artifact_id` uses the value that was generated after you registered model files as artifacts.

In [18]:
# Replace the artifact_id field value with your own ID, then execute the code.
from ai_core_sdk.models import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "watsonx-demo-serve-conf",
    resource_group = "default",
    scenario_id = "watsonx-demo",
    executable_id = "watsonx-demo-serve-23091515",
    input_artifact_bindings = [
        # list
        InputArtifactBinding(key="modelArtifact", artifact_id=artifact_id)
    ]
)

print(response.__dict__)
conf_id=response.__dict__['id']

{'id': 'f2ae995b-bd14-4ea9-97c3-31637bacc3ce', 'message': 'Configuration created'}


<a id="start_deployment"></a>
### Step 9: Start the deployment

Replace the `configuration_id` field value with the ID from the previous step. Execute the code:

In [19]:
response = ai_core_client.deployment.create(
    resource_group = "default",
    configuration_id = conf_id
)
print(response.__dict__)
deploy_id=response.__dict__['id']

{'id': 'deb874467896b396', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


<div class="alert alert-block alert-warning">
<b>Important:</b>

Note the unique ID generated of your deployment. You may create multiple deployments using the same configuration ID, each of which will have a unique endpoint.

</div>

Check deployment status:

In [20]:
while str(response.status)!='Status.RUNNING':

    response = ai_core_client.deployment.get(
        resource_group = 'default',
        deployment_id = deploy_id
    )

    print("Status: ", response.status)
    print('*'*80)
    time.sleep(30)

print(response.__dict__)

Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
*******************************

<div class="alert alert-block alert-info">

<b>Tip:</b>

This may take a few minutes for deployment status to change: `UNKNOWN` > `PENDING` > `RUNNING`.

</div>

Now query the deployment logs to view its output. Replace with the `deployment_id` field value with your own ID, and execute.

In [21]:
response = ai_core_client.deployment.query_logs(
    resource_group = "default",
    deployment_id = deploy_id
)

for log in response.data.result:
    print(log.msg)
    print("---")

INFO:root:Initializing, args: src_uri [s3://sap-ai-core-demo-bucket/watsonx/demo/models] dest_path[ [/mnt/models]
---
INFO:root:Copying contents of s3://sap-ai-core-demo-bucket/watsonx/demo/models to local
---
INFO:botocore.credentials:Found credentials in environment variables.
---
INFO:root:Successfully copied s3://sap-ai-core-demo-bucket/watsonx/demo/models to /mnt/models
---
[STARTING RUNTIME]
---
{'log_code': '<COR25991305W>', 'message': 'No library version found', 'args': None}
---
---
---
---
---
---
---
---
.
---
.
---
.
---
.
---
.
---
.
---
.
---
.
---
{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<COM11997772I>", "message": "Loading service module: generated.common_service_pb2", "num_indent": 0, "thread_id": 140004906088256, "timestamp": "2024-02-17T23:26:22.720606"}
---
{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<COM11997772I>", "message": "Loading service module: generated.common_service_pb2_grpc", "num_inden

<a id="make_prediction"></a>
### Step 10: Make a prediction

You can now use the URL from your model deployment to make inference calls to your model. The `custom_endpoint` is generated by the code pieces inside of the docker image. This is customizable and you can add multiple routes for different purposes. 

Replace the `deployment_id` field value with your deployment ID and execute the code.

In [22]:
# Make an inference call using SAP AI Core SDK

query_text = "DEPENDS ON CHEMICAL COMPOSITION; FISSION PRODUCTS SAFETY GLASSES EYE WASH STATION. APRONS. SPECIAL IMPERVIOUS CLOTHING. IMPERVIOUS SCBA IF INVOLVED IN FIRE, OTHERWISE GAS MASK. PROVIDE MECHAN(GEN/LOCAL EXHAUST)VENT TO MAINTN <TLV"
deployment_id = deploy_id

custom_endpoint = "/v1/watson.runtime.nlp.v1/NlpService/ClassificationPredict"

prediction = ai_core_client.rest_client.post(
    resource_group = "default",
    path = "/inference/deployments/" + deployment_id + custom_endpoint,
    headers = {
        "grpc-metadata-mm-model-id": "msds_classification_model",
        "Content-Type": "application/json"},
    body = {
        "rawDocument": {
            "text": query_text
        },
        "languageCode": "en"
    }
)

print(prediction)

{'classes': [{'class_name': 'WEAR CHEMICAL PROTECTIVE SUIT WITH SELF CONTD BRTHG APP.', 'confidence': 0.9815408}, {'class_name': 'WEAR NIOSH/MSHA APPROVED SCBA & FULL PROTECTIVE EQUIPMENT .', 'confidence': 0.007698022}, {'class_name': 'NONE', 'confidence': 0.007614372}, {'class_name': 'WEAR NIOSH/MSHA APPROVED SCBA AND FULL PROTECTIVE EQUIPMENT .', 'confidence': 0.0058819777}, {'class_name': 'FIRE FIGHTERS SHOULD USE NIOSH APPROVED SCBA & FULL PROTECTIVE EQUIPMENT WHEN FIGHTING CHEMICAL FIRE. USE WATER SPRAY TO COOL NEARBY CONTAINERS EXPOSED TO FIRE.', 'confidence': 0.0054885778}, {'class_name': 'SELF-CONTAINED BREATHING APPARATUS.', 'confidence': 0.003914192}, {'class_name': 'COMBUSTION PRODUCTS OF THIS MATERIAL MAY BE TOXIC. USE A SELF-CONTAINED BREATHING APPARATUS WHEN FIGHTING FIRES INVOLVING CHEMICAL PRODUCTS.', 'confidence': 0.0021899275}, {'class_name': 'USE NIOSH APPROVED SCBA AND FULL PROTECTIVE EQUIPMENT .', 'confidence': 0.0015990753}, {'class_name': 'WATER SPRAY MAY BE INEF

In [23]:
# Make an inference call using Requests library, but still using SAP AI Core SDK to get a token

import requests

# URL found in the deployment status
deployment_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/" + deploy_id

# Preparing the input for inference
query_text = "DEPENDS ON CHEMICAL COMPOSITION; FISSION PRODUCTS SAFETY GLASSES EYE WASH STATION. APRONS. SPECIAL IMPERVIOUS CLOTHING. IMPERVIOUS SCBA IF INVOLVED IN FIRE, OTHERWISE GAS MASK. PROVIDE MECHAN(GEN/LOCAL EXHAUST)VENT TO MAINTN <TLV"
body = {
    "rawDocument": {
        "text": query_text
    },
    "languageCode": "en"
}

endpoint = f"{deployment_url}/v1/watson.runtime.nlp.v1/NlpService/ClassificationPredict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default",
           "grpc-metadata-mm-model-id": "msds_classification_model",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=body)

print('Inference result:', response.json())

Inference result: {'classes': [{'className': 'WEAR CHEMICAL PROTECTIVE SUIT WITH SELF CONTD BRTHG APP.', 'confidence': 0.9815408}, {'className': 'WEAR NIOSH/MSHA APPROVED SCBA & FULL PROTECTIVE EQUIPMENT .', 'confidence': 0.007698022}, {'className': 'NONE', 'confidence': 0.007614372}, {'className': 'WEAR NIOSH/MSHA APPROVED SCBA AND FULL PROTECTIVE EQUIPMENT .', 'confidence': 0.0058819777}, {'className': 'FIRE FIGHTERS SHOULD USE NIOSH APPROVED SCBA & FULL PROTECTIVE EQUIPMENT WHEN FIGHTING CHEMICAL FIRE. USE WATER SPRAY TO COOL NEARBY CONTAINERS EXPOSED TO FIRE.', 'confidence': 0.0054885778}, {'className': 'SELF-CONTAINED BREATHING APPARATUS.', 'confidence': 0.003914192}, {'className': 'COMBUSTION PRODUCTS OF THIS MATERIAL MAY BE TOXIC. USE A SELF-CONTAINED BREATHING APPARATUS WHEN FIGHTING FIRES INVOLVING CHEMICAL PRODUCTS.', 'confidence': 0.0021899275}, {'className': 'USE NIOSH APPROVED SCBA AND FULL PROTECTIVE EQUIPMENT .', 'confidence': 0.0015990753}, {'className': 'WATER SPRAY MA

<a id="stop_deployment"></a>
### Step 11: Stop the deployment

A running deployment incurs cost because it is allocated cloud resources. Stopping the deployment frees up these resources and therefore there is no charge for a deployment of status `Stopped`.

Replace with the `deployment_id` field value with your ID, and execute the code, to stop the deployment.



In [25]:
from ai_core_sdk.models import TargetStatus

response = ai_core_client.deployment.modify(
    resource_group = "default",
    deployment_id = deploy_id,
    target_status = TargetStatus.STOPPED
)

print(response.__dict__)

{'id': 'd2bafeb14802e155', 'message': 'Deployment modification scheduled'}


<div class="alert alert-block alert-info">

<b>Tip:</b>

You cannot restart a deployment. You must create a new deployment, reusing the configuration. Each deployment will have a different URL.
</div>

<a id="summary"></a>
## Summary

Now we've demonstrated a complete end-to-end flow, in which a Watson NLP model is first trained in Watson Studio, with training dataset stored in SAP Datasphere and securely transferred over, and then the trained model is securely deployed to SAP AI Core as a serving workflow managed through GitOps.